# Step 1 - Install the required dependencies and make sure the python version is 3.10 and above

# Step 2 - Load a dataset from Hugging Face

In [1]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("cardiffnlp/tweet_eval", "sentiment")
df = pd.DataFrame(ds['test']).head(500)
df.head(5)

,text,label
0,@user @user what do these '1/2 naked pics' hav...,1
1,OH: “I had a blue penis while I was this” [pla...,1
2,"@user @user That's coming, but I think the vic...",1
3,I think I may be finally in with the in crowd ...,2
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",0


In [2]:
def label_map(x):
    if x == 0:
        return 'negative'
    elif x == 1:
        return 'neutral'
    elif x == 2:
        return 'positive'
    return x
df['label'] = df['label'].map(label_map)

# Step 3 - Run model inference

Warning: This step is going to download two models of ~500MB each. 

**If you don't want to download the models, you can jump to step 4 and use the provided data in the repo instead.**

### Run inference with roberta

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", 
                model="cardiffnlp/twitter-roberta-base-sentiment-latest",
                model_kwargs={"device_map":0}
               )

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [4]:
import tqdm

results = []
texts = df['text'].to_list()

## Depending on your machine, this should take around 1 minute
for text in tqdm.tqdm(texts):
    results.append(pipe(text))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset                           | 7/500 [00:01<00:57,  8.59it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 62.12it/s]


In [5]:
df['roberta'] = [r[0]['label'] for r in results]
df['roberta_score'] = [r[0]['score'] for r in results]

### Run inference with gpt2

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", 
                model="LYTinn/finetuning-sentiment-model-tweet-gpt2",
                model_kwargs={"device_map": 0}
               )

Device set to use cuda:0


In [7]:
import tqdm

results = []
texts = df['text'].to_list()

## Depending on your machine, this should take around 1 minute
for text in tqdm.tqdm(texts):
    results.append(pipe(text))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 69.19it/s]


In [8]:
df['gpt2'] = [r[0]['label'] for r in results]
df['gpt2_score'] = [r[0]['score'] for r in results]

## map labels back
def label_map(x):
    if x == 'LABEL_0':
        return 'negative'
    elif x == 'LABEL_1':
        return 'neutral'
    elif x == 'LABEL_2':
        return 'positive'
    return x
df['gpt2'] = df['gpt2'].map(label_map)

# Step 4 - Pre-processing data and add additional columns

In [9]:
## If you skip the model inference, uncomment the code below and load the provided data

# df = pd.read_csv('tweets.csv')

In [10]:
df["input_length"] = df["text"].str.len()

# Step 5 - Start Zeno for interactive slicing

In this step, you need to create 5 slices in the Zeno interface and derive meaningful insights.

As a starting point, try to create the two slices we provide:

1. Tweets with hashtags
2. Tweets with strong positive words (e.g., love) -- you can determine the exact words

Creating slices in Zeno is straightforward: Just click on the '+' button for 'create a new slice', and you can define the slice using existing column attributes, with simple value macthing or even regular expression.

![image.png](images/image.png)

There are more fun features in Zeno, including interactive metadata & model comparison -- feel free to check the teaser video in [README](https://github.com/zeno-ml/zeno) of the Zeno repository.

In [ ]:
df["index"] = df.index.tolist()

In [18]:
## If you don't have a Zeno account already, create one on Zeno Hub (https://hub.zenoml.com/signup). 
## After logging in to Zeno Hub, generate your API key by clicking on your profile at the top right to navigate to your account page.

# !pip install zeno_client
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd

# df = pd.read_csv('tweets.csv')
# df = df.reset_index()

# Initialize a client with the API key.
client = ZenoClient("your_key")

project = client.create_project(
    name="V2_Tweet Sentiment Analysis",
    view="text-classification",
    metrics=[
        ZenoMetric(name="accuracy", type="mean", columns=["correct"]),
    ]
)

project.upload_dataset(df, id_column="index", data_column="text", label_column="label")

models = ['roberta', 'gpt2']
for model in models:
    df_system = df[['index', model]]
    
    # Measure accuracy for each instance, which is averaged by the ZenoMetric above
    df_system["correct"] = (df_system[model] == df["label"]).astype(int)
    
    project.upload_system(df_system, name=model, id_column="index", output_column=model)

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/b31de552-b483-4cd1-9f97-b551531039a0/V2_Tweet%20Sentiment%20Analysis


C:\Users\santy\miniconda3\envs\desp_ia\lib\site-packages\zeno_client\util.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57'
 '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71'
 '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85'
 '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99'
 '100' '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111'
 '112' '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123'
 '124' '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135'
 '136' '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147'
 '148' '149'

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded data


C:\Users\santy\AppData\Local\Temp\ipykernel_17076\1564226634.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_system["correct"] = (df_system[model] == df["label"]).astype(int)


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


C:\Users\santy\AppData\Local\Temp\ipykernel_17076\1564226634.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_system["correct"] = (df_system[model] == df["label"]).astype(int)


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


In [19]:
df

,text,label,roberta,roberta_score,gpt2,gpt2_score,input_length,index
0,@user @user what do these '1/2 naked pics' hav...,neutral,negative,0.804726,positive,0.913452,96,0
1,OH: “I had a blue penis while I was this” [pla...,neutral,neutral,0.866949,neutral,0.753407,72,1
2,"@user @user That's coming, but I think the vic...",neutral,neutral,0.763724,positive,0.999962,87,2
3,I think I may be finally in with the in crowd ...,positive,positive,0.774047,positive,0.898783,83,3
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,neutral,0.416397,positive,0.986431,133,4
...,...,...,...,...,...,...,...,...
495,Donnie is gonna deport the Menendez Brothers r...,negative,negative,0.915344,neutral,0.999632,69,495
496,Still reading #SettleForMore @user #fridayreads,neutral,neutral,0.797208,positive,0.999972,47,496
497,#Chocolate cupcake #candle melting with its sw...,positive,positive,0.951857,negative,0.999672,96,497
498,Is this leaf ?Can I eat ?Open the leaf!Oh! Thi...,neutral,positive,0.664862,negative,0.999548,101,498


After running the code above, you should be able to access Zeno in http://localhost:8231


After successfully creating the two slices, come up with three *additional* slices you want to check and **create** the slices in the Zeno interface.

There are two directions to identify useful slices:
- Top-down: Think about what kinds of things the model can struggle with, and come up with some slices.
- Bottom-up: Look at model (mis-)predictions, come up with hypotheses, and translate them into data slices.

3. [YOUR CHOICE]
4. [YOUR CHOICE]
5. [YOUR CHOICE]

In [ ]:
## Write down descriptions of additional slices you created

custom_slice_descriptions = [
    "",
]

# Step 6 - Write down three addition data slices you want to create but do not have the metadata for slicing

In the previous step, you might have already come up with some slices you wanted to create but found it hard to do with existing metadata. Write down three of such slices in this step.

Example: 
- I want to create a slice on tweets using slangs
- I want to create a slice on non-English tweets (if any)

In [ ]:
## Write down three additional data slices here:

additional_slice_descriptions = [
    "",
]

# Step 7 - Generate more test cases with Large Language Models

Select one slice from the three you wrote down and generate **10 test cases** using LLMs, which can include average case, boundary case, or difficult case.

Your input can be in the following format:

> Examples:
> - OH: “I had a blue penis while I was this” [playing with Google Earth VR]
> - @user @user That’s coming, but I think the victims are going to be Medicaid recipients.
> - I think I may be finally in with the in crowd #mannequinchallenge  #grads2014 @user
> 
> Generate more tweets using slangs.

The first part of **Examples** conditions the LLM on the style, length, and content of examples. The second part of **Instructions** instructs what kind of examples you want LLM to generate.

Use our provided GPTs to start the task: [llm-based-test-case-generator](https://chatgpt.com/g/g-982cylVn2-llm-based-test-case-generator). If you do not have access to GPTs, use the plain ChatGPT or other LLM providers you have access to instead.

In [20]:
## Write down the slice you select

slice_description = "Tweets con violencia de género"

## Write down all generated test cases here

generated_test_cases = [
"Women should know their place.",

"She's just asking for it with the way she dresses.",

"Why would a woman want to be in charge? She'll only make a mess.",

"Girls are only good for looking pretty, not for serious work.",

"A woman is only valuable when she’s quiet and obedient.",

"She’s too emotional to make important decisions.",

"Women are just drama queens who can't take a joke.",

"Why do women always have to complain about everything?",

"She was asking for trouble when she went out like that.",

"It's hard to take a woman seriously when she's crying.",
]

In [21]:
generated_test_cases

['Women should know their place.',
 "She's just asking for it with the way she dresses.",
 "Why would a woman want to be in charge? She'll only make a mess.",
 'Girls are only good for looking pretty, not for serious work.',
 'A woman is only valuable when she’s quiet and obedient.',
 'She’s too emotional to make important decisions.',
 "Women are just drama queens who can't take a joke.",
 'Why do women always have to complain about everything?',
 'She was asking for trouble when she went out like that.',
 "It's hard to take a woman seriously when she's crying."]

In [23]:
df_nuevos = pd.DataFrame({'text': generated_test_cases})

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   text           500 non-null    object 
 1   label          500 non-null    object 
 2   roberta        500 non-null    object 
 3   roberta_score  500 non-null    float64
 4   gpt2           500 non-null    object 
 5   gpt2_score     500 non-null    float64
 6   input_length   500 non-null    int64  
 7   index          500 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 31.4+ KB
